In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

scSpark   = SparkSession.builder.appName("reading csv").getOrCreate()

In [2]:
data_file = 'olist_customers_dataset.csv'
customer_data = scSpark.read.csv(data_file, header=True, sep=",").cache()
    
state = customer_data.groupBy('customer_state').count()
print(state.show())

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SC| 3637|
|            RO|  253|
|            PI|  495|
|            AM|  148|
|            RR|   46|
|            GO| 2020|
|            TO|  280|
|            MT|  907|
|            SP|41746|
|            ES| 2033|
|            PB|  536|
|            RS| 5466|
|            MS|  715|
|            AL|  413|
|            MG|11635|
|            PA|  975|
|            BA| 3380|
|            SE|  350|
|            PE| 1652|
|            CE| 1336|
+--------------+-----+
only showing top 20 rows

None


In [3]:
customer_data.registerTempTable("customers")
output =  scSpark.sql("SELECT * from customers WHERE customer_state = 'SP' AND customer_city='sao paulo'")
output.show()

+--------------------+--------------------+------------------------+-------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+--------------------+--------------------+------------------------+-------------+--------------+
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|    sao paulo|            SP|
|fd826e7cf63160e53...|addec96d2e059c80c...|                   04534|    sao paulo|            SP|
|eabebad39a88bb6f5...|295c05e81917928d7...|                   05704|    sao paulo|            SP|
|9b8ce803689b3562d...|7f3a72e8f988c6e73...|                   05416|    sao paulo|            SP|
|2938121a40a20953c...|482441ea6a06b1f72...|                   05713|    sao paulo|            SP|
|cb721d7b4f271fd87...|a5844ba4bfc8d0cc6...|                   08225|    sao paulo|            SP|
|6e359a57a91f84095...|2e6a42a9b5cbb0da6...|                   04571|    sao paulo|            SP|
|e3109970a3fe8021d..

In [4]:
output.write.format('json').save('customers_data.json')

In [5]:
scSpark = SparkSession.builder.appName("reading csv").config("spark.driver.extraClassPath", "C:\spark\spark\jars\mysql-connector-java-5.1.48.jar").getOrCreate()

In [8]:
output = scSpark.sql('SELECT customer_state, customer_city, COUNT(*) as Total from customers GROUP BY customer_state, customer_city')
#output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://localhost:3306/ecommerce',
    driver='com.mysql.cj.jdbc.Driver',
    dbtable='customer_data',
    user='root',
    password='Gurukrupa104#').mode('append').save()